## Creación de coordenadas y consulta a API para matriz de distancias

In [78]:
import requests
import pandas as pd


#Se lee el archivo de coordenadas
coords = pd.read_csv("coords.csv", encoding ="latin1")
lcoords = []

#Se crea la lista de coordenadas
for i in range(len(coords)):
    lcoords.append([coords.iloc[i]['lon'],coords.iloc[i]['lat']])


#Se consulta en la API OpenRouteServices la matriz de distancias 
body = {"locations":lcoords,"metrics":["distance"]}
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248ca4dee32a47c4427af5864e37bedccd1',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

#Para consultar la matriz correr la siguiente linea.
#call.text

## Método heurístico

In [80]:
#creo matriz de distancias a partir de lo obtenido por la API OpenRouteServices
dmatriz = [[0.0,7577.79,9753.97,18957.71,20390.06,15382.47,13128.53,6260.45,11889.59,10046.79,6990.56,14693.79,43360.36,6338.18,19522.57,12755.98],[6970.54,0.0,11118.56,15562.4,13875.43,11987.16,4606.2,3912.2,7316.1,10143.35,9362.34,11298.48,36845.72,5635.12,20887.16,9360.67],[10460.94,11655.47,0.0,17959.79,24467.75,15620.26,13366.32,8759.21,12127.37,9864.7,14631.28,13695.86,47438.04,6186.73,11400.12,11957.2],[16614.23,14863.04,17308.75,0.0,27056.93,7845.45,13356.12,12815.46,12117.18,8546.97,20805.85,4505.11,50027.22,13163.27,25169.44,7946.41],[19912.61,13582.41,24060.63,27575.8,0.0,21107.48,14361.16,16854.27,17071.06,22785.69,18084.97,23311.88,32498.39,18577.2,33829.24,19096.39],[14769.62,11452.09,15231.3,7689.75,21350.73,0.0,6931.52,9404.51,6410.98,8581.32,17394.9,6705.35,44321.02,9513.06,23092.0,4146.8],[12642.16,6311.96,15599.17,13739.3,14138.57,7174.64,0.0,7234.97,2775.13,8949.19,15225.36,9475.38,37108.86,7343.52,23459.87,5259.89],[5662.6,3923.44,7854.12,13128.71,16735.71,9682.33,7428.39,0.0,6189.45,7712.72,9798.74,8864.78,39706.0,2369.69,17622.73,7055.84],[11668.98,5613.35,12030.04,12616.31,15575.93,6632.53,1875.12,6093.56,0.0,7826.2,14083.96,8352.38,38546.22,6202.11,22336.88,4136.9],[9793.15,9371.89,9599.0,10038.65,21877.14,7522.57,8176.34,7324.31,6937.39,0.0,15314.71,5774.73,44847.44,5307.55,17459.69,4036.07],[6910.2,8243.35,13229.51,21437.78,17105.35,17862.54,10893.56,8740.51,14369.65,13522.34,0.0,17173.85,40075.64,9813.72,18685.27,15236.04],[13430.74,10674.0,13349.14,5753.68,23097.32,6647.54,9396.51,8626.42,8157.57,5429.08,16616.81,0.0,46067.61,8631.03,21209.83,3986.8],[41126.96,34796.76,45274.98,48790.16,30834.45,42321.83,35575.52,38068.63,38285.42,44000.05,39299.32,44526.23,0.0,39791.55,55043.59,40310.75],[5258.14,6225.11,5826.76,13636.27,19037.39,10189.9,7935.96,3328.85,6697.01,5265.13,10105.91,9372.35,42007.68,0.0,15595.37,7563.41],[18873.09,20067.63,9920.46,25403.69,32879.9,22887.61,21778.47,17171.36,20539.53,16833.29,23242.23,21139.76,55850.19,14598.89,0.0,19401.1],[11778.55,8221.75,11496.66,8314.85,18823.0,3827.25,5122.2,6174.17,3883.25,4846.68,14164.57,3454.69,41793.3,6521.99,19357.35,0.0]]

#Transformo la matriz en DataFrame (no necesario, solo más visual)
dfdistances = pd.DataFrame(data=dmatriz)
dfdistances.replace(0.00, 999999, inplace=True) #valores de camino a sí mismo se convierten en 999999 para evitar loops
dfdistancesC = dfdistances.copy() #Copia del df, en uno se consulta el camino y se modifica, en el otro se consultan los valores originales


#Se se inicializan variables (primer estado en 0)
costototal = 0
camino = [0]
estado = 0
sumadecostos = []

#Greedy, siempre se agarra el camino más corto, al temrinar se regresa al 0
for i in range(len(dfdistances)):
    idxmin = list(dfdistances.iloc[estado]).index(min(dfdistances.iloc[estado]))
    costototal += dfdistancesC.iloc[estado][idxmin]
    sumadecostos.append(dfdistancesC.iloc[estado][idxmin])
    dfdistances[estado] = 999999
    estado = idxmin
    camino.append(estado)

## Consulta de Resultados

In [81]:
print(costototal)
print(sumadecostos)
print(camino)

190080.43999999997
[6260.45, 2369.69, 5265.13, 4036.07, 3454.69, 5753.68, 7845.45, 6410.98, 1875.12, 6311.96, 9362.34, 13229.51, 11400.12, 32879.9, 32498.39, 41126.96]
[0, 7, 13, 9, 15, 11, 3, 5, 8, 6, 1, 10, 2, 14, 4, 12, 0]
